In [2]:
import numpy as np
import pandas as pd 
import os
import open3d as o3d
import matplotlib.pyplot as plt

/Users/osvaldohernandez/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Data preprocessing

#### Data Loading
In this section, three main datasets are loaded:
- df_images: Contains the image data for object detection, including file paths for image_id, image_name, and corresponding metadata.
- df_labels: Contains the ground truth labels for object detection, including object_class, bounding_box_coordinates, and image_id.
- df_calibration: Includes calibration parameters such as camera matrix, distortion coefficients, and extrinsic parameters for aligning image and sensor data.

In [3]:
image_path  = "./data_object_image_2/training/image_2/"

In [4]:
# List all image files
image_files = os.listdir(image_path)

# Create a DataFrame to store image file paths
df_images = pd.DataFrame({
    'image_id': range(len(image_files)),
    'image_name': image_files,
    'file_path': [os.path.join(image_path, img) for img in image_files]
})

In [5]:
# Print the first few entries of df_images
print(df_images.head())

   image_id  image_name                                          file_path
0         0  004863.png  ./data_object_image_2/training/image_2/004863.png
1         1  006912.png  ./data_object_image_2/training/image_2/006912.png
2         2  006906.png  ./data_object_image_2/training/image_2/006906.png
3         3  004877.png  ./data_object_image_2/training/image_2/004877.png
4         4  005599.png  ./data_object_image_2/training/image_2/005599.png


In [6]:
# Define the path to your KITTI dataset labels
label_path = "./training/label_2/"

# Function to load and parse a label file
def load_labels(label_file):
    labels = []
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            obj_class = parts[0]
            x_min = int(float(parts[4]))
            y_min = int(float(parts[5]))
            x_max = int(float(parts[6]))
            y_max = int(float(parts[7]))
            labels.append({'object_class': obj_class, 'bounding_box': (x_min, y_min, x_max, y_max)})
    return labels

# List all label files and accumulate label data
label_files = os.listdir(label_path)
label_data = []

for i, label_file in enumerate(label_files):
    file_path = os.path.join(label_path, label_file)
    labels = load_labels(file_path)
    for label in labels:
        label_data.append({
            'image_id': i,
            'object_class': label['object_class'],
            'bounding_box_coordinates': label['bounding_box']
        })

# Create a DataFrame using pd.concat
df_labels = pd.DataFrame(label_data)

# Print the first few entries of df_labels
print(df_labels.head())

   image_id object_class bounding_box_coordinates
0         0          Car     (595, 177, 637, 217)
1         0          Car       (0, 185, 198, 277)
2         0          Car    (812, 169, 1023, 293)
3         0          Car     (764, 172, 947, 261)
4         0          Car     (708, 177, 852, 243)


Check the shape of each DataFrame

In [7]:
print("Images DataFrame shape:", df_images.shape)
print("Labels DataFrame shape:", df_labels.shape)

Images DataFrame shape: (7481, 3)
Labels DataFrame shape: (51865, 3)


Check for missing values in each DataFrame

In [8]:
print("Missing values in Images DataFrame:\n", df_images.isna().sum())
print("Missing values in Labels DataFrame:\n", df_labels.isna().sum())

Missing values in Images DataFrame:
 image_id      0
image_name    0
file_path     0
dtype: int64
Missing values in Labels DataFrame:
 image_id                    0
object_class                0
bounding_box_coordinates    0
dtype: int64


Get a summary of the data types and basic info

In [9]:
print("Images DataFrame info:\n")
df_images.info()

print("Labels DataFrame info:\n")
df_labels.info()

Images DataFrame info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7481 entries, 0 to 7480
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_id    7481 non-null   int64 
 1   image_name  7481 non-null   object
 2   file_path   7481 non-null   object
dtypes: int64(1), object(2)
memory usage: 175.5+ KB
Labels DataFrame info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51865 entries, 0 to 51864
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   image_id                  51865 non-null  int64 
 1   object_class              51865 non-null  object
 2   bounding_box_coordinates  51865 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [10]:
# Function to read point cloud data from a KITTI .bin file
def read_point_cloud_from_bin(bin_file):
    point_cloud = np.fromfile(bin_file, dtype=np.float32).reshape(-1, 4)
    return point_cloud

In [16]:
# Load a point cloud file (adjust the path to your .bin file)
bin_file = "./data_object_velodyne/training/velodyne/000001.bin"
point_cloud = read_point_cloud_from_bin(bin_file)

# Create an Open3D PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud[:, :3])  # Use only x, y, z (ignore intensity)

# Create a visualizer window
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)

# Non-blocking visualization with a way to exit
while vis.poll_events():
    vis.update_geometry(pcd)
    vis.update_renderer()

# Destroy the window after exiting the loop
vis.destroy_window()

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


KeyboardInterrupt: 

: 

In [12]:
image_path  = "./data_object_image_2/training/image_2/"

In [13]:
# List all image files
image_files = os.listdir(image_path)

# Create a DataFrame to store image file paths
df_images = pd.DataFrame({
    'image_id': range(len(image_files)),
    'image_name': image_files,
    'file_path': [os.path.join(image_path, img) for img in image_files]
})

In [14]:
# Print the first few entries of df_images
print(df_images.head())

   image_id  image_name                                          file_path
0         0  004863.png  ./data_object_image_2/training/image_2/004863.png
1         1  006912.png  ./data_object_image_2/training/image_2/006912.png
2         2  006906.png  ./data_object_image_2/training/image_2/006906.png
3         3  004877.png  ./data_object_image_2/training/image_2/004877.png
4         4  005599.png  ./data_object_image_2/training/image_2/005599.png


In [15]:
# Define the path to your KITTI dataset labels
label_path = "./data_object_label_2/training/label_2/"

# Function to load and parse a label file
def load_labels(label_file):
    labels = []
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            obj_class = parts[0]
            x_min = int(float(parts[4]))
            y_min = int(float(parts[5]))
            x_max = int(float(parts[6]))
            y_max = int(float(parts[7]))
            labels.append({'object_class': obj_class, 'bounding_box': (x_min, y_min, x_max, y_max)})
    return labels

# List all label files and accumulate label data
label_files = os.listdir(label_path)
label_data = []

for i, label_file in enumerate(label_files):
    file_path = os.path.join(label_path, label_file)
    labels = load_labels(file_path)
    for label in labels:
        label_data.append({
            'image_id': i,
            'object_class': label['object_class'],
            'bounding_box_coordinates': label['bounding_box']
        })

# Create a DataFrame using pd.concat
df_labels = pd.DataFrame(label_data)

# Print the first few entries of df_labels
print(df_labels.head())

FileNotFoundError: [Errno 2] No such file or directory: './data_object_label_2/training/label_2/'

In [ ]:
# Define the path to your KITTI dataset calibration files
calibration_path = "./data_object_calib/training/calib/"

# Function to load calibration parameters
def load_calibration(calib_file):
    calib_data = {}
    with open(calib_file, 'r') as f:
        for line in f:
            line = line.strip()
            
            # Skip empty lines or comments
            if not line:
                continue

            # Check if the line has the expected ':' delimiter
            if ':' in line:
                key, value = line.split(':', 1)
                calib_data[key.strip()] = [float(x) for x in value.strip().split()]
            else:
                print(f"Skipping malformed line: {line}")
    
    return calib_data

# List all calibration files and accumulate calibration data
calibration_files = os.listdir(calibration_path)
calibration_data = []

for i, calib_file in enumerate(calibration_files):
    file_path = os.path.join(calibration_path, calib_file)
    calib_data = load_calibration(file_path)
    calibration_data.append({
        'image_id': i,
        'calibration_data': calib_data
    })

# Create a DataFrame using pd.concat
df_calibration = pd.DataFrame(calibration_data)

# Print the first few entries of df_calibration
print(df_calibration.head())

   image_id                                   calibration_data
0         0  {'P0': [707.0493, 0.0, 604.0814, 0.0, 0.0, 707...
1         1  {'P0': [721.5377, 0.0, 609.5593, 0.0, 0.0, 721...
2         2  {'P0': [721.5377, 0.0, 609.5593, 0.0, 0.0, 721...
3         3  {'P0': [721.5377, 0.0, 609.5593, 0.0, 0.0, 721...
4         4  {'P0': [721.5377, 0.0, 609.5593, 0.0, 0.0, 721...


Check the shape of each DataFrame

In [ ]:
print("Images DataFrame shape:", df_images.shape)
print("Labels DataFrame shape:", df_labels.shape)
print("Calibration DataFrame shape:", df_calibration.shape)

Images DataFrame shape: (7481, 3)
Labels DataFrame shape: (51865, 3)
Calibration DataFrame shape: (7481, 2)


Check for missing values in each DataFrame

In [ ]:
print("Missing values in Images DataFrame:\n", df_images.isna().sum())
print("Missing values in Labels DataFrame:\n", df_labels.isna().sum())
print("Missing values in Calibration DataFrame:\n", df_calibration.isna().sum())

Missing values in Images DataFrame:
 image_id      0
image_name    0
file_path     0
dtype: int64
Missing values in Labels DataFrame:
 image_id                    0
object_class                0
bounding_box_coordinates    0
dtype: int64
Missing values in Calibration DataFrame:
 image_id            0
calibration_data    0
dtype: int64


Get a summary of the data types and basic info

In [ ]:
print("Images DataFrame info:\n")
df_images.info()

print("Labels DataFrame info:\n")
df_labels.info()

print("Calibration DataFrame info:\n")
df_calibration.info()

Images DataFrame info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7481 entries, 0 to 7480
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_id    7481 non-null   int64 
 1   image_name  7481 non-null   object
 2   file_path   7481 non-null   object
dtypes: int64(1), object(2)
memory usage: 175.5+ KB
Labels DataFrame info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51865 entries, 0 to 51864
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   image_id                  51865 non-null  int64 
 1   object_class              51865 non-null  object
 2   bounding_box_coordinates  51865 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB
Calibration DataFrame info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7481 entries, 0 to 7480
Data columns (total 2 columns):
 #   Column            Non-Null